In [38]:
from youtube_transcript_api import YouTubeTranscriptApi
from urllib.parse import urlparse, parse_qs

def get_video_id(url):
    parsed = urlparse(url)

    if parsed.hostname in ["www.youtube.com", "youtube.com"]:
        return parse_qs(parsed.query).get("v", [None])[0]

    if parsed.hostname == "youtu.be":
        return parsed.path[1:]

    return None


# DÁN FULL LINK YOUTUBE Ở ĐÂY
youtube_url = "https://www.youtube.com/watch?v=wc9O-9mcObc"

video_id = get_video_id(youtube_url)

if not video_id:
    print("❌ Không lấy được video ID từ link này")
else:
    try:
        # API mới: dùng fetch()
        transcript = YouTubeTranscriptApi().fetch(video_id)

        full_text = " ".join([item.text for item in transcript])
        print(full_text)

    except Exception as e:
        print("❌ Lỗi khi lấy transcript:", e)


Hello, I'm Zach. Today I want to share with 
you a new development paradigm for large   language model application. So I recently built 
a YouTube summarization tool that uses a large   language model to help you extract interesting 
topics and explain the questions and answers   in a very friendly way. Now, the best part here 
is that I built the whole thing in just 1 hour,   and so can you. The secret source behind it is 
pocket flow, a large language model framework in   just 100 lines of code. You heard that right—it's 
just 100 lines. But the simplicity is its power,   because it allows AI assistants like cursor AI to 
build the application for you. So in this video,   I will show you a step-by-step tutorial on 
how I built such a YouTube summarization   application using cursor AI plus pocket 
flow in just an hour. Let's dive right in. So let me start by quickly motivating why I built 
this YouTube summarization tool. Lex just dropped   a very nice podcast a few weeks ago on the 

In [39]:
import yt_dlp
import requests
import re

def get_youtube_transcript_clean(video_url):
    ydl_opts = {
        'skip_download': True,        # Không tải video, chỉ lấy metadata
        'writesubtitles': True,       # Lấy phụ đề do người dùng tải lên
        'writeautomationsub': True,   # Lấy phụ đề tự động
        'sub_langs': ['vi', 'en'],    # Ưu tiên tiếng Việt, sau đó là tiếng Anh
        'quiet': True,
        'no_warnings': True
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(video_url, download=False)
            
            # 1. Xác định ngôn ngữ và lấy URL phụ đề
            subs = info.get('subtitles') or info.get('automatic_captions')
            
            if not subs:
                return "Video không có phụ đề."

            # Chọn ngôn ngữ khả dụng
            lang = 'vi' if 'vi' in subs else ('en' if 'en' in subs else list(subs.keys())[0])
            
            # Lấy URL định dạng json3 (đây là định dạng dễ lấy text nhất)
            sub_url = None
            for sub in subs[lang]:
                if sub.get('ext') == 'json3' or 'json3' in sub.get('url', ''):
                    sub_url = sub['url']
                    break
            
            if not sub_url:
                sub_url = subs[lang][0]['url'] # Fallback nếu không thấy json3

            # 2. Tải nội dung phụ đề
            response = requests.get(sub_url)
            data = response.json()

            # 3. Trích xuất text từ định dạng JSON3 của YouTube
            full_text = []
            for event in data.get('events', []):
                if 'segs' in event:
                    for seg in event['segs']:
                        text = seg.get('utf8', '').strip()
                        if text:
                            full_text.append(text)

            return " ".join(full_text)

    except Exception as e:
        return f"Lỗi: {str(e)}"

# Chạy thử
url = "https://www.youtube.com/watch?v=wc9O-9mcObc" # Thay bằng URL của bạn
result = get_youtube_transcript_clean(url)

print("--- TRANSCRIPT HOÀN CHỈNH ---")
print(result)

# Lưu vào file
with open("transcript_ytdlp.txt", "w", encoding="utf-8") as f:
    f.write(result)

--- TRANSCRIPT HOÀN CHỈNH ---
Hello, I'm Zach. Today I want to share with 
you a new development paradigm for large language model application. So I recently built 
a YouTube summarization tool that uses a large language model to help you extract interesting 
topics and explain the questions and answers in a very friendly way. Now, the best part here 
is that I built the whole thing in just 1 hour, and so can you. The secret source behind it is 
pocket flow, a large language model framework in just 100 lines of code. You heard that right—it's 
just 100 lines. But the simplicity is its power, because it allows AI assistants like cursor AI to 
build the application for you. So in this video, I will show you a step-by-step tutorial on 
how I built such a YouTube summarization application using cursor AI plus pocket 
flow in just an hour. Let's dive right in. So let me start by quickly motivating why I built 
this YouTube summarization tool. Lex just dropped a very nice podcast a few weeks

In [41]:
import yt_dlp
import os
import json
from pathlib import Path

class YouTubeDownloader:
    def __init__(self, output_dir='./youtube_downloads'):
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        
    def get_video_info(self, url):
        """Lấy thông tin chi tiết về video"""
        ydl_opts = {
            'quiet': True,
            'no_warnings': True,
            'skip_download': True,
        }
        
        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(url, download=False)
                
                # Thông tin cơ bản
                video_info = {
                    'title': info.get('title', 'Unknown'),
                    'channel': info.get('uploader', 'Unknown'),
                    'duration': info.get('duration', 0),
                    'views': info.get('view_count', 0),
                    'upload_date': info.get('upload_date', 'Unknown'),
                    'description': info.get('description', '')[:500] + '...' if info.get('description') else '',
                    'thumbnail': info.get('thumbnail', ''),
                    'url': url,
                    'formats': []
                }
                
                # Lấy danh sách các định dạng có sẵn
                for fmt in info.get('formats', []):
                    format_info = {
                        'format_id': fmt.get('format_id', ''),
                        'ext': fmt.get('ext', ''),
                        'resolution': fmt.get('resolution', ''),
                        'fps': fmt.get('fps', 0),
                        'filesize': fmt.get('filesize', 0),
                        'vcodec': fmt.get('vcodec', ''),
                        'acodec': fmt.get('acodec', ''),
                    }
                    video_info['formats'].append(format_info)
                
                return video_info
                
        except Exception as e:
            print(f"Lỗi khi lấy thông tin: {e}")
            return None
    
    def download_video(self, url, quality='best', download_type='video'):
        """
        Tải video/audio với nhiều tùy chọn
        
        Parameters:
        url (str): URL YouTube
        quality (str): 'best', 'worst', '1080p', '720p', '480p', '360p'
        download_type (str): 'video', 'audio', 'both'
        """
        try:
            # Cấu hình dựa trên loại tải
            ydl_opts = {
                'outtmpl': os.path.join(self.output_dir, '%(title)s.%(ext)s'),
                'progress_hooks': [self.progress_hook],
                'noplaylist': True,
                'quiet': False,
            }
            
            if download_type == 'audio':
                # Chỉ tải audio
                ydl_opts.update({
                    'format': 'bestaudio/best',
                    'postprocessors': [{
                        'key': 'FFmpegExtractAudio',
                        'preferredcodec': 'mp3',
                        'preferredquality': '192',
                    }],
                })
            elif download_type == 'video':
                # Tải video
                if quality == 'best':
                    ydl_opts['format'] = 'bestvideo+bestaudio/best'
                elif quality == 'worst':
                    ydl_opts['format'] = 'worstvideo+worstaudio/worst'
                else:
                    # Lọc theo độ phân giải
                    ydl_opts['format'] = f'bestvideo[height<={quality.replace("p", "")}]+bestaudio/best[height<={quality.replace("p", "")}]'
                
                ydl_opts['merge_output_format'] = 'mp4'
                
            elif download_type == 'both':
                # Tải cả video và audio riêng
                ydl_opts['format'] = 'bestvideo,bestaudio'
            
            # Tải
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                print("Đang tải...")
                ydl.download([url])
                
        except Exception as e:
            print(f"Lỗi khi tải: {e}")
    
    def progress_hook(self, d):
        """Hiển thị tiến trình tải"""
        if d['status'] == 'downloading':
            percent = d.get('_percent_str', 'N/A').strip()
            speed = d.get('_speed_str', 'N/A').strip()
            eta = d.get('_eta_str', 'N/A').strip()
            print(f"\rĐang tải: {percent} | Tốc độ: {speed} | Còn lại: {eta}", end='')
        elif d['status'] == 'finished':
            print(f"\n✓ Hoàn thành!")
    
    def download_playlist(self, playlist_url, limit=None):
        """Tải toàn bộ playlist"""
        try:
            ydl_opts = {
                'outtmpl': os.path.join(self.output_dir, '%(playlist_title)s', '%(playlist_index)s - %(title)s.%(ext)s'),
                'format': 'bestvideo+bestaudio/best',
                'merge_output_format': 'mp4',
                'ignoreerrors': True,  # Bỏ qua lỗi nếu có video không tải được
            }
            
            if limit:
                ydl_opts['playlistend'] = limit
            
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                print("Đang tải playlist...")
                ydl.download([playlist_url])
                
        except Exception as e:
            print(f"Lỗi khi tải playlist: {e}")
    
    def download_subtitles(self, url, languages=['en', 'vi']):
        """Tải phụ đề"""
        try:
            ydl_opts = {
                'outtmpl': os.path.join(self.output_dir, '%(title)s.%(ext)s'),
                'writesubtitles': True,
                'writeautomaticsub': True,
                'subtitleslangs': languages,
                'skip_download': True,  # Chỉ tải phụ đề, không tải video
            }
            
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                print("Đang tải phụ đề...")
                ydl.download([url])
                
        except Exception as e:
            print(f"Lỗi khi tải phụ đề: {e}")
    
    def list_available_formats(self, url):
        """Hiển thị tất cả định dạng có sẵn"""
        ydl_opts = {
            'listformats': True,
            'quiet': True,
        }
        
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            print(f"\nĐịnh dạng có sẵn cho: {url}")
            ydl.download([url])

def main_menu():
    """Menu chính"""
    downloader = YouTubeDownloader()
    
    while True:
        print("\n" + "="*50)
        print("YouTube Downloader với yt-dlp")
        print("="*50)
        print("1. Tải video")
        print("2. Tải chỉ audio (MP3)")
        print("3. Tải playlist")
        print("4. Tải phụ đề")
        print("5. Xem thông tin video")
        print("6. Xem định dạng có sẵn")
        print("7. Thoát")
        print("="*50)
        
        choice = input("Chọn chức năng (1-7): ").strip()
        
        if choice == '7':
            print("Tạm biệt!")
            break
        
        url = input("Nhập URL YouTube: ").strip()
        
        if choice == '1':
            print("\nChọn chất lượng:")
            print("1. Tốt nhất")
            print("2. 1080p")
            print("3. 720p")
            print("4. 480p")
            print("5. 360p")
            qual_choice = input("Chọn (1-5): ").strip()
            
            quality_map = {
                '1': 'best',
                '2': '1080p',
                '3': '720p',
                '4': '480p',
                '5': '360p'
            }
            
            quality = quality_map.get(qual_choice, 'best')
            downloader.download_video(url, quality=quality, download_type='video')
            
        elif choice == '2':
            downloader.download_video(url, download_type='audio')
            
        elif choice == '3':
            limit = input("Số lượng video tối đa (Enter để tải tất cả): ").strip()
            limit = int(limit) if limit else None
            downloader.download_playlist(url, limit=limit)
            
        elif choice == '4':
            langs = input("Ngôn ngữ phụ đề (cách nhau bởi dấu phẩy, ví dụ: en,vi): ").strip()
            languages = [lang.strip() for lang in langs.split(',')] if langs else ['en']
            downloader.download_subtitles(url, languages=languages)
            
        elif choice == '5':
            info = downloader.get_video_info(url)
            if info:
                print(f"\nThông tin video:")
                print(f"Tiêu đề: {info['title']}")
                print(f"Kênh: {info['channel']}")
                print(f"Thời lượng: {info['duration']} giây")
                print(f"Lượt xem: {info['views']:,}")
                print(f"Ngày upload: {info['upload_date']}")
                print(f"Mô tả: {info['description']}")
                
        elif choice == '6':
            downloader.list_available_formats(url)
            
        else:
            print("Lựa chọn không hợp lệ!")

# ==================== SỬ DỤNG ====================

if __name__ == "__main__":
    # Chạy menu chính
    main_menu()
    
    # Hoặc sử dụng trực tiếp
    # downloader = YouTubeDownloader()
    
    # # Tải video chất lượng tốt nhất
    # downloader.download_video("https://www.youtube.com/watch?v=wc9O-9mcObc")
    
    # # Tải chỉ audio (MP3)
    # downloader.download_video("https://www.youtube.com/watch?v=wc9O-9mcObc", download_type='audio')
    
    # # Tải playlist
    # downloader.download_playlist("https://www.youtube.com/playlist?list=PL...")
    
    # # Lấy thông tin video
    # info = downloader.get_video_info("https://www.youtube.com/watch?v=wc9O-9mcObc")
    # print(json.dumps(info, indent=2, ensure_ascii=False))


YouTube Downloader với yt-dlp
1. Tải video
2. Tải chỉ audio (MP3)
3. Tải playlist
4. Tải phụ đề
5. Xem thông tin video
6. Xem định dạng có sẵn
7. Thoát

Thông tin video:
Tiêu đề: Build an AI YouTube Summarizer in 1 hour - Here's My Secret Framework | Pocketflow
Kênh: Zachary Huang
Thời lượng: 776 giây
Lượt xem: 11,495
Ngày upload: 20250302
Mô tả: *Pocket Flow:* https://github.com/The-Pocket/PocketFlow
*AI YouTube Summarizer:* https://github.com/The-Pocket/Tutorial-Youtube-Made-Simple
*Pocket Flow Python Project Template:* https://github.com/The-Pocket/PocketFlow-Template-Python
*LLM Application Development Playbook:* https://the-pocket.github.io/PocketFlow/guide.html
*Map Reduce Design Pattern:* https://github.com/The-Pocket/PocketFlow/blob/main/docs/design_pattern/mapreduce.md

*Outline:*
0:00 Introduction
0:51 Project Motivation
2:30 I...

YouTube Downloader với yt-dlp
1. Tải video
2. Tải chỉ audio (MP3)
3. Tải playlist
4. Tải phụ đề
5. Xem thông tin video
6. Xem định dạng có sẵn
7.

ERROR: [generic] '6' is not a valid URL


Lỗi khi lấy thông tin: ERROR: [generic] '6' is not a valid URL

YouTube Downloader với yt-dlp
1. Tải video
2. Tải chỉ audio (MP3)
3. Tải playlist
4. Tải phụ đề
5. Xem thông tin video
6. Xem định dạng có sẵn
7. Thoát

Định dạng có sẵn cho: https://www.youtube.com/watch?v=wc9O-9mcObc


ID      EXT   RESOLUTION FPS CH |   FILESIZE   TBR PROTO | VCODEC        VBR ACODEC      ABR ASR MORE INFO
----------------------------------------------------------------------------------------------------------------------------------------------------------------
sb3     mhtml 48x27        0    |                  mhtml | images                                storyboard
sb2     mhtml 80x45        0    |                  mhtml | images                                storyboard
sb1     mhtml 160x90       0    |                  mhtml | images                                storyboard
sb0     mhtml 320x180      0    |                  mhtml | images                                storyboard
139-drc m4a   audio only      2 |    4.51MiB   49k https | audio only        mp4a.40.5   49k 22k [en-US] English (US) original (default), low, DRC, m4a_dash
249-drc webm  audio only      2 |    4.71MiB   51k https | audio only        opus        51k 48k [en-US] English (US) original (default), low, 